In [82]:
import pandas as pd
import numpy as np
import statistics
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score,mean_absolute_error
import lightgbm as lgbm

In [44]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,AdaBoostRegressor,ExtraTreesRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.linear_model import ElasticNet

In [20]:
df = pd.read_csv('train_folds.csv')
df_test = pd.read_csv('test.csv')

In [21]:
df.sample(4)

,Company,TypeName,Ram,Weight,Price,Touchscreen,IPS,pipi,cpu_brand,HDD,SSD,Hybrid,Flash_Storage,Gpu_brand,os,kfold
277,HP,Notebook,6,2.10,28185,0,0,141.211998,AMD Processor,1000,0,0,0,AMD,Windows,1
583,Lenovo,2 in 1 Convertible,4,1.80,33992,1,1,157.350512,Intel Core i7,0,128,0,0,Intel,Windows,2
913,Lenovo,Notebook,8,1.65,71661,0,0,157.350512,Intel Core i5,0,256,0,0,Intel,Windows,3
587,HP,Notebook,4,1.86,13746,0,0,100.454670,AMD Processor,500,0,0,0,AMD,Others/Linux/No_OS,2


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1041 entries, 0 to 1040
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Company        1041 non-null   object 
 1   TypeName       1041 non-null   object 
 2   Ram            1041 non-null   int64  
 3   Weight         1041 non-null   float64
 4   Price          1041 non-null   int64  
 5   Touchscreen    1041 non-null   int64  
 6   IPS            1041 non-null   int64  
 7   pipi           1041 non-null   float64
 8   cpu_brand      1041 non-null   object 
 9   HDD            1041 non-null   int64  
 10  SSD            1041 non-null   int64  
 11  Hybrid         1041 non-null   int64  
 12  Flash_Storage  1041 non-null   int64  
 13  Gpu_brand      1041 non-null   object 
 14  os             1041 non-null   object 
 15  kfold          1041 non-null   int64  
dtypes: float64(2), int64(9), object(5)
memory usage: 130.2+ KB


In [23]:
df['Company'].value_counts()

Lenovo       243
Dell         239
HP           222
Asus         119
Acer          83
Toshiba       41
MSI           37
Apple         17
Other         16
Mediacom       7
Razer          6
Samsung        6
Microsoft      5
Name: Company, dtype: int64

In [24]:
for fold in range(4):
    
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)
    
    y_train = df_train.Price.values
    y_valid = df_valid.Price.values
    
    df_train = df_train.drop(columns=['Price'])
    df_train = df_train.drop(columns=['kfold'])
    df_valid = df_valid.drop(columns=['Price'])
    df_valid = df_valid.drop(columns=['kfold'])
    
    df_valid = df_valid[df_train.columns]
    
    X_train = df_train.copy()
    X_valid = df_valid.copy()
    
    step1 = ColumnTransformer(transformers=[
        ('col_tnf',OneHotEncoder(sparse=False,drop='first'),[0,1,7,12,13])
    ],remainder='passthrough')
    
    step2 = LinearRegression()
    
    pipe = Pipeline([
        ('step1',step1),
        ('step2',step2)
    ])
    
    pipe.fit(X_train,y_train)
    
    y_pred = pipe.predict(X_valid)
    
    print('R2 score',r2_score(y_valid,y_pred))
    print('MAE',mean_absolute_error(y_valid,y_pred))

R2 score 0.776068746594273
MAE 13367.987068965518
R2 score 0.7667020691603048
MAE 12808.638461538461
R2 score 0.7870696508461621
MAE 12577.859615384616
R2 score 0.7187389155832546
MAE 13558.096153846154


In [32]:
for fold in range(4):
    
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)
    
    y_train = np.log(df_train['Price'])
    y_valid = np.log(df_valid['Price'])
    
    df_train = df_train.drop(columns=['Price'])
    df_train = df_train.drop(columns=['kfold'])
    df_valid = df_valid.drop(columns=['Price'])
    df_valid = df_valid.drop(columns=['kfold'])
    
    df_valid = df_valid[df_train.columns]
    
    X_train = df_train.copy()
    X_valid = df_valid.copy()
    
    step1 = ColumnTransformer(transformers=[
        ('col_tnf',OneHotEncoder(sparse=False,drop='first'),[0,1,7,12,13])
    ],remainder='passthrough')
    
    step2 = LinearRegression()
    
    pipe = Pipeline([
        ('step1',step1),
        ('step2',step2)
    ])
    
    pipe.fit(X_train,y_train)
    
    y_pred = pipe.predict(X_valid)
    
    print('R2 score',r2_score(y_valid,y_pred))
    print('MAE',mean_absolute_error(y_valid,y_pred))

R2 score 0.804739094443048
MAE 0.22643203612950827
R2 score 0.8327488360793723
MAE 0.20357906954997315
R2 score 0.8228952196015934
MAE 0.20776537486858082
R2 score 0.8109342317428495
MAE 0.20542575123531684


In [83]:
for fold in range(4):
    
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)
    
    y_train = np.log(df_train['Price'])
    y_valid = np.log(df_valid['Price'])
    
    df_train = df_train.drop(columns=['Price','kfold'])
    df_valid = df_valid.drop(columns=['Price','kfold'])
    
    df_valid = df_valid[df_train.columns]
    
    X_train = df_train.copy()
    X_valid = df_valid.copy()
    
    step1 = ColumnTransformer(transformers=[
        ('col_tnf',OneHotEncoder(sparse=False,drop='first'),[0,1,7,12,13])
    ],remainder='passthrough')
    
    step2 = lgbm.LGBMRegressor()
    
    pipe = Pipeline([
        ('step1',step1),
        ('step2',step2)
    ])
    
    pipe.fit(X_train,y_train)
    
    y_pred = pipe.predict(X_valid)
    
    print('R2 score',r2_score(y_valid,y_pred))
    print('MAE',mean_absolute_error(y_valid,y_pred))

R2 score 0.8551361366620978
MAE 0.18213875557462886
R2 score 0.8928073811944689
MAE 0.15940298684562917
R2 score 0.8949710977718132
MAE 0.1588625802105061
R2 score 0.8418905986654387
MAE 0.18874061532374034


In [84]:
for fold in range(4):
    
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)
    
    y_train = np.log(df_train['Price'])
    y_valid = np.log(df_valid['Price'])
    
    df_train = df_train.drop(columns=['Price'])
    df_train = df_train.drop(columns=['kfold'])
    df_valid = df_valid.drop(columns=['Price'])
    df_valid = df_valid.drop(columns=['kfold'])
    
    df_valid = df_valid[df_train.columns]
    
    X_train = df_train.copy()
    X_valid = df_valid.copy()
    
    step1 = ColumnTransformer(transformers=[
        ('col_tnf',OneHotEncoder(sparse=False,drop='first'),[0,1,7,12,13])
    ],remainder='passthrough')
    
    step2 = RandomForestRegressor(n_estimators=100,
                              random_state=3,
                              max_samples=0.5,
                              max_features=0.75,
                              max_depth=15)
    
    pipe = Pipeline([
        ('step1',step1),
        ('step2',step2)
    ])
    
    pipe.fit(X_train,y_train)
    
    y_pred = pipe.predict(X_valid)
    
    print('R2 score',r2_score(y_valid,y_pred))
    print('MAE',mean_absolute_error(y_valid,y_pred))

R2 score 0.8503088831041224
MAE 0.18368200932255663
R2 score 0.8944423749107583
MAE 0.15943930733825792
R2 score 0.87812579926005
MAE 0.17599278713263716
R2 score 0.869659081278936
MAE 0.17450950256179806


In [75]:
models = []
models.append(('LR' , LinearRegression()))
models.append(('RIDGE', Ridge(alpha=10)))
models.append(('LASSO' , Lasso(alpha=0.001)))
models.append(('EN' , ElasticNet()))
models.append(('KNN' , KNeighborsRegressor(n_neighbors=3)))
models.append(('CART' , DecisionTreeRegressor()))
models.append(('SVR' , SVR(kernel='rbf',C=10000,epsilon=0.1)))
models.append(('DT', DecisionTreeRegressor(max_depth=8)))
models.append(('RF', RandomForestRegressor(n_estimators=100,
                              random_state=3,
                              max_features=0.75,
                              max_depth=15)))
models.append(('ET',ExtraTreesRegressor(n_estimators=100,
                              random_state=3,
                              max_features=0.75,
                              max_depth=15)))
# models.append(('ADB',AdaBoostRegressor(n_estimators=15,learning_rate=1.0)))
# models.append(('GDB',GradientBoostingRegressor(n_estimators=500)))
# models.append(('XGB',XGBRegressor(n_estimators=45,max_depth=5,learning_rate=0.5)))

In [76]:
print(models)

[('LR', LinearRegression()), ('RIDGE', Ridge(alpha=10)), ('LASSO', Lasso(alpha=0.001)), ('EN', ElasticNet()), ('KNN', KNeighborsRegressor(n_neighbors=3)), ('CART', DecisionTreeRegressor()), ('SVR', SVR(C=10000)), ('DT', DecisionTreeRegressor(max_depth=8)), ('RF', RandomForestRegressor(max_depth=15, max_features=0.75, random_state=3)), ('ET', ExtraTreesRegressor(max_depth=15, max_features=0.75, random_state=3))]


In [77]:
r2_score_result = []
mae_result = []
names = []
for name, model in models:
    for fold in range(4):
        df_train = df[df.kfold != fold].reset_index(drop=True)
        df_valid = df[df.kfold == fold].reset_index(drop=True)
        y_train = np.log(df_train['Price'])
        y_valid = np.log(df_valid['Price'])
        df_train = df_train.drop(columns=['Price'])
        df_train = df_train.drop(columns=['kfold'])
        df_valid = df_valid.drop(columns=['Price'])
        df_valid = df_valid.drop(columns=['kfold'])
        df_valid = df_valid[df_train.columns]
        X_train = df_train.copy()
        X_valid = df_valid.copy()
        step1 = ColumnTransformer(transformers=[
            ('col_tnf',OneHotEncoder(sparse=False,drop='first'),[0,1,7,12,13])
        ],remainder='passthrough')

        step2 = model

        pipe = Pipeline([
            ('step1',step1),
            ('step2',step2)
        ])

        pipe.fit(X_train,y_train)

        y_pred = pipe.predict(X_valid)
        
        r2_score_calc = r2_score(y_valid,y_pred)
        mae_calc = mean_absolute_error(y_valid,y_pred)
        
    r2_score_result.append(r2_score_calc)
    mae_result.append(mae_calc)
    names.append(name)
    msg = "%s %f (%f)" % (name,statistics.mean(r2_score_result),statistics.mean(mae_result))
    print(msg)

LR 0.810934 (0.205426)
RIDGE 0.807739 (0.207789)
LASSO 0.807849 (0.207646)
EN 0.738130 (0.244402)
KNN 0.743149 (0.241533)
CART 0.748531 (0.239190)
SVR 0.750814 (0.235725)
DT 0.759018 (0.231608)
RF 0.771658 (0.224831)
ET 0.778207 (0.220819)


In [81]:
final_predictions = []
for fold in range(4):
    
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)
    
    y_train = np.log(df_train['Price'])
    y_valid = np.log(df_valid['Price'])
    
    df_train = df_train.drop(columns=['Price','kfold'])
    df_valid = df_valid.drop(columns=['Price','kfold'])
    
    df_valid = df_valid[df_train.columns]
    
    X_train = df_train.copy()
    X_valid = df_valid.copy()
    X_test = df_test.copy()
    
    step1 = ColumnTransformer(transformers=[
        ('col_tnf',OneHotEncoder(sparse=False,drop='first'),[0,1,7,12,13])
    ],remainder='passthrough')
    
    step2 = LinearRegression()
    
    pipe = Pipeline([
        ('step1',step1),
        ('step2',step2)
    ])
    
    pipe.fit(X_train,y_train)
    
    y_pred = pipe.predict(X_valid)
    
    print('R2 score',r2_score(y_valid,y_pred))
    print('MAE',mean_absolute_error(y_valid,y_pred))
    
    test_pred = pipe.predict(X_test)
    
    final_predictions.append(test_pred)

R2 score 0.804739094443048
MAE 0.22643203612950827
R2 score 0.8327488360793723
MAE 0.20357906954997315
R2 score 0.8228952196015934
MAE 0.20776537486858082
R2 score 0.8109342317428495
MAE 0.20542575123531684


In [ ]:
# import pickle

pickle.dump(df, open('df.pkl,'wb))
pickle.dump(df, open('pipe.pkl,'wb))